Chrome version: 130.0.6723.70
Chrome Driver Path: /usr/local/bin

sudo /Applications/Google\ Chrome.app/Contents/MacOS/Google\ Chrome --remote-debugging-port=9223 --user-data-dir='~/ChromeProfile'

### 0. Setting

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from tqdm import tqdm


import pandas as pd
import numpy as np

import pickle
import time
import re

In [2]:
# pd.set_option('display.max_rows',None)

In [3]:
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9221")
chrome_driver = "/Applications/Google\ Chrome.app/Contents/MacOS/Google"

In [4]:
services = ChromeService(executable_path = chrome_driver)
driver = webdriver.Chrome(options=chrome_options)

The chromedriver version (130.0.6723.69) detected in PATH at /usr/local/bin/chromedriver might not be compatible with the detected chrome version (131.0.6778.109); currently, chromedriver 131.0.6778.108 is recommended for chrome 131.*, so it is advised to delete the driver in PATH and retry


### 2. 각 회사의 Review 데이터 수집

##### 0. Settings

In [5]:
link_event_time = 5
click_event_time = 3

##### 1. 회사 검색 및 리뷰창 이동

In [6]:
def get_search_url(company_name):
    
    """ Search Company and Get Review page """

    base_url = 'https://www.glassdoor.com/Reviews/index.htm?overall_rating_low=3.5&page=1&locId=1147401&locType=C&occ=Data%20Scientist'
    driver.get(base_url)
    time.sleep(link_event_time)

    try:
        # Locate and input the company name in the search bar
        search_xpath = '//*[@id="companyAutocomplete-companyDiscover-employerSearch"]'
        search_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, search_xpath)))
        search_element.send_keys(company_name)
        time.sleep(click_event_time)

        # Click search button
        button_xpath = '//*[@id="Explore"]/div[2]/div/div/div[2]/button'
        driver.find_element(By.XPATH, button_xpath).click()
        time.sleep(click_event_time)

        # Click first result link
        search_list_xpath = '//*[@id="Discover"]/div/div/div[1]/div[1]/div[1]'
        first_result = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, search_list_xpath))
        )
        first_result_link = first_result.find_elements(By.CSS_SELECTOR, 'a')[0].get_attribute('href')
        driver.get(first_result_link)
        time.sleep(click_event_time)

        # Go to reviews page
        a_element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '#reviews > a')))
        review_href = a_element.get_attribute('href')
        return review_href
        
    except Exception as e:
        print(f"Error navigating to search URL for {company_name}: {e}")
        return None

##### 2. 마지막 Review page 탐색

In [7]:
def get_last_page():

    """ Get Last Review page"""
    
    try:
        last_p_elements = driver.find_elements(By.CSS_SELECTOR, "p.pagination_PageNumberText__zy_hr")
        return int([p.text for p in last_p_elements][-1])
        
    except Exception as e:
        print(f"Error finding last page: {e}")
        return 1

##### 3. Review 요소 수집

In [8]:
def extract_review_details(li_element):
    
    """ Extract Review Deatails """
    
    review_dict = {}
    try:
        ############################# 각 요소 ############################# 
        date = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'timestamp_reviewDate__dsF9n'))).text
        rating = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-rating_ratingLabel__0_Hk9'))).text
        one_line_review = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'heading_Heading__BqX5J.heading_Level3__X81KK'))).text
        job = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-avatar_avatarLabel__P15ey'))).text
        history = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'text-with-icon_TextWithIcon__5ZZqT'))).text
        
        try:
            region = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-avatar_tagsContainer__9NCNs'))).text.split('\n')[1]
        except:
            region=None
        
        pros = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-test="review-text-PROS"]'))).text
        cons = WebDriverWait(li_element, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '[data-test="review-text-CONS"]'))).text
    
        try:
            show_more_button_class_nm = 'expand-button_ExpandButton__Wevvg'
            show_more_button = WebDriverWait(li_element, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, show_more_button_class_nm))
            )
            show_more_button.click()
    
            feedback = WebDriverWait(li_element, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '[data-test="review-text-FEEDBACK"]'))
            ).text
        except:
            feedback = None
        
        review_dict.update({
            'Date': date,
            'Job': job,
            'History': history,
            'Region': region if region else None,
            'One_line_review': one_line_review,
            'Rating': rating,
            'Pros': pros,
            'Cons': cons,
            'Feedback': feedback# if feedback else None,
        })

        ############################# Sub Rating ############################# 
        try:
            rating_element = li_element.find_element(By.CLASS_NAME, 'review-details-bar_reviewRatingAndFeaturedContainer__J8iG9')
            time.sleep(2)
            
            # ActionChains로 마우스를 이동시켜서 숨겨진 요소 표시
            ActionChains(driver).move_to_element(rating_element).perform()
            tooltip = WebDriverWait(driver, 5).until(
                EC.visibility_of_element_located((By.CSS_SELECTOR, '[data-test="review-subratings-tooltip"]'))
            )
            # 숨겨진 요소의 요소 전부 값 가져오기
            tooltip_html = tooltip.get_attribute("innerHTML")
            soup = BeautifulSoup(tooltip_html, 'html.parser')
            
            # 각 항목을 포함하는 div를 찾음
            sub_ratings = soup.find_all('div', class_='review-rating_subRating__0Q_Z0')
            
            # 각 항목별로 점수 계산
            for rating in sub_ratings:
                # 항목 이름 가져오기
                category = rating.find('span', class_='review-rating_subRatingText__Wn3AL').text.strip()
                # --outline-percentage가 0%인 별만 카운트
                stars = len(rating.find_all('div', style="--outline-percentage: 0%;"))# 별 개수를 점수로 계산
                review_dict[category] = stars
                
        except Exception:
            pass;

        ############################# Check List #############################
        try:
            check_list = li_element.find_elements(By.CLASS_NAME, 'review-details_experienceContainer__2W06X')

            # 각 요소의 전체 HTML 출력
            for element in check_list:
                html_content = element.get_attribute("outerHTML")
                soup = BeautifulSoup(html_content, 'lxml')

                # 클래스 마지막 부분에 따른 값 매핑 딕셔너리
                class_value_map = {
                    'LGHYG': 'V',
                    'BCbQd': 'empty',
                    'jmOo8': '-',
                    '0GqA8': 'X'
                }

                # 각 span 요소의 텍스트와 해당 클래스명 끝자리에 따른 값 저장
                for span in soup.find_all('span'):
                    # span 텍스트 가져오기
                    span_text = span.text.strip()
                    # 부모 div 요소의 마지막 클래스명 부분 가져오기
                    parent_div = span.find_parent('div')
                    last_class_part = parent_div['class'][-1].split('_')[-1]
                    # 매핑된 값을 result 딕셔너리에 추가
                    review_dict[span_text] = class_value_map.get(last_class_part, 'Unknown')
        
        except Exception:
            pass;
    
    except Exception as e:
        print(f"Error extracting review details: {e}")
    
    return review_dict

##### 4. Filtering

In [9]:
# Filtering 함수
def apply_filtering(driver):

    """ Filtering Clear """
    
    try:
        # Filtering 버튼 상위 div XPath
        parent_div_xpath = '//*[@id="__next"]/div[2]/div/main/div/div[6]/div[4]/div[1]'#/div'
        
        # 상위 div 요소 찾기
        parent_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, parent_div_xpath))
        )
        # 상위 div 내 하위 버튼 찾기
        filtering_button = parent_div.find_element(By.TAG_NAME, 'button')
        filtering_button.click()
        time.sleep(5)
        # filtering_button = WebDriverWait(parent_div, 10).until(
        #     EC.element_to_be_clickable((By.TAG_NAME, 'button'))
        # )
        # filtering_button.click()

        # Clear 버튼 상위 div XPath
        parent_div_clear_path = '//*[@id="__next"]/div[2]/div/main/div/div[6]/div[4]/div[1]/div/div/div'
        parent_clear_div = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, parent_div_clear_path))
        )
        # 상위 div 내 하위 버튼 찾기
        clear_button_class_nm = 'button_Button__MlD2g.button-base_Button__knLaX'
        clear_button = parent_clear_div.find_element(By.CLASS_NAME, clear_button_class_nm)

        # ActionChains로 Clear 버튼 클릭
        actions = ActionChains(driver)
        actions.move_to_element(clear_button).click().perform()
        time.sleep(5)
        
    except Exception as e:
        print(f"Error during button click process: {e}")

In [10]:
# Filtering 함수
def apply_filtering(driver):

    """ Filtering Clear """

    parent_div_xpath = '//*[@id="__next"]/div[2]/div/main/div/div[6]/div[4]/div[1]'#/div'
    
    # 상위 div 요소 찾기
    parent_div = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, parent_div_xpath))
    )
    # 상위 div 내 하위 버튼 찾기
    filtering_button = parent_div.find_element(By.TAG_NAME, 'button')
    filtering_button.click()
    time.sleep(5)

    # Clear 버튼 상위 div XPath
    parent_div_clear_path = '//*[@id="__next"]/div[2]/div/main/div/div[6]/div[4]/div[1]/div/div/div'
    parent_clear_div = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, parent_div_clear_path))
    )
    # 상위 div 내 하위 버튼 찾기
    clear_button_class_nm = 'button_Button__MlD2g.button-base_Button__knLaX'
    clear_button = parent_clear_div.find_element(By.CLASS_NAME, clear_button_class_nm)

    # ActionChains로 Clear 버튼 클릭
    actions = ActionChains(driver)
    actions.move_to_element(clear_button).click().perform()
    time.sleep(5)

##### 5. 데이터 수집

In [11]:
def scrape_reviews(company_list):
    global review_df, result_df, complete_company_list, filtering_fail_list
    review_df = pd.DataFrame()
    result_df = pd.DataFrame()
    # complete_company =[]
    # filtering_fail_list=[]
    
    for company in company_list:
        print(f"Processing company: {company}")
        file_name = f"reviews_{company}.csv"
        # complete_company_list.append(company)
        
        review_origin_url = get_search_url(company)
        driver.get(review_origin_url)

        try:
            WebDriverWait(driver, link_event_time).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))  # 예: 'body' 태그가 로드될 때까지 대기
            )
        except TimeoutException:
            print("페이지 로드가 시간 내 완료되지 않았습니다.")

        # Filtering 처리
        try:
            apply_filtering(driver)
        except:
            with open('filtering_fail_list.pickle', 'rb') as f: # Read
                filtering_fail_list = pickle.load(f)
            filtering_fail_list.append(company) # Append
            
            with open("filtering_fail_list.pickle","wb") as f: # Write
                pickle.dump(filtering_fail_list, f)
            print('filterling button not found')
            
        # last page search
        last_page = get_last_page()
        print('last page:', last_page)

        iteration_count = 0
        for page in tqdm(range(1, last_page + 1)):
            iteration_count+=1
            
            if page > 1:
                review_url = review_origin_url.replace('.htm', f'_P{page}.htm')
                driver.get(review_url)
                time.sleep(link_event_time)
            else:
                # driver.get(review_origin_url)
                # time.sleep(link_event_time)
                pass;

            try:
                li_elements = driver.find_elements(By.XPATH, '//*[@id="ReviewsFeed"]/ol/li')
                for li_element in li_elements:
                    review_dict = extract_review_details(li_element)
                    review_dict['Company'] = company
                    review_df = pd.concat([review_df, pd.DataFrame([review_dict])], ignore_index=True)

                if iteration_count%100:
                    review_df.to_csv(f'tmp_company_review/{file_name}', index=False)
                    
            except Exception as e:
                print(f"Error processing reviews on page {page} for {company}: {e}")
        
        result_df = pd.concat([result_df, review_df], ignore_index=True)
        result_df.to_csv('company_review_data/complete_'+file_name, index=False)
        
        with open('complete_company_list.pickle', 'rb') as f:
            complete_company_list = pickle.load(f)
        complete_company_list.append(company)
        
        with open("complete_company_list.pickle","wb") as f:
            pickle.dump(complete_company_list, f)
        
        print(f"{company} complete !")

    print("Scraping complete")
    return result_df

In [12]:
company_df = pd.read_csv('company_review_summary.csv')
company_df = company_df[company_df['review']>500].sort_values('review').reset_index(drop=True)
company_list = company_df['company'].to_list()

In [13]:
with open('complete_company_list.pickle', 'rb') as f:
    complete_company_list = pickle.load(f)
extra_companies = [company for company in company_list if company not in complete_company_list]

In [14]:
extra_companies
# company_list

['Lithia Motors, Inc.',
 'Inter-American Development Bank',
 'Opendoor Technologies, Inc.',
 'Valvoline, Inc.',
 'News Corp.',
 'The New York Times Co.',
 'Ingersoll Rand, Inc.',
 'Teck Resources Limited',
 'Zoetis, Inc.',
 'Mettler-Toledo International, Inc.',
 'Mohawk Industries, Inc.',
 'Guidewire Software, Inc.',
 'Equity Residential',
 'Nasdaq, Inc.',
 'Hasbro, Inc.',
 'Sensata Technologies Holding Plc',
 'LKQ Corp.',
 'Sun Life Financial, Inc.',
 'Polaris Inc.',
 'Kinder Morgan, Inc.',
 'Ontario Power Generation, Inc.',
 'Celestica, Inc.',
 'TD SYNNEX Corp.',
 'Avantor, Inc.',
 'Xylem, Inc.',
 'Ball Corp.',
 'American Tower Corp.',
 'Bio-Rad Laboratories, Inc.',
 'Waters Corp.',
 'Crown Castle, Inc.',
 'Dentsply Sirona, Inc.',
 'AvalonBay Communities, Inc.',
 'Public Storage',
 'Regeneron Pharmaceuticals, Inc.',
 'Lam Research Corp.',
 'Trimble, Inc.',
 'Avnet, Inc.',
 'BorgWarner, Inc.',
 'AutoNation, Inc.',
 'Henry Schein, Inc.',
 'Canadian National Railway Co.',
 'Air Products

In [ ]:
company_review_df = scrape_reviews(extra_companies)
print("All reviews saved.")

Processing company: Lithia Motors, Inc.
last page: 88


 34%|████████████▉                         | 30/88 [2:48:02<5:51:18, 363.42s/it]